## **A) Importações**

In [1]:
#pip install jupyter-dash

In [2]:
#pip install dash

In [213]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib as plt
%matplotlib inline

import plotly.express as px
import plotly.graph_objects as go
import plotly.colors as colors

from dash import dash, dcc, html, callback_context
from dash.dependencies import Input, Output, State
import dash.exceptions
import dash_table

from jupyter_dash import JupyterDash

In [2]:
df1 = pd.read_csv("variaveis_totais.csv")
df2 = pd.read_excel("eventos_totais.xlsx")

#### **A.1) Countries info**

In [3]:
df1.head()

,Unnamed: 0,country,country_code,indicator,indicator_user,year,value
0,0,Albania,ALB,gdp_growth,GDP Growth (%),1960,NaN
1,1,Albania,ALB,gdp_growth,GDP Growth (%),1961,NaN
2,2,Albania,ALB,gdp_growth,GDP Growth (%),1962,NaN
3,3,Albania,ALB,gdp_growth,GDP Growth (%),1963,NaN
4,4,Albania,ALB,gdp_growth,GDP Growth (%),1964,NaN


In [4]:
var = df1[['country','country_code','indicator', 'indicator_user', 'year','value']]

In [5]:
# Encontre os anos mínimos e máximos no dataframe
min_year = var.year.min()
max_year = var.year.max()

#### **A.2) Events info**

In [6]:
df2 = df2[['year_event','country_event','event_type','event']]

In [7]:
ev = df2.query('year_event > 1960')

In [8]:
ev.head()

,year_event,country_event,event_type,event
81,1961,Germany,Geopolitical,Construction began on the Berlin Wall between ...
82,1961,Malta,Geopolitical,The State of Malta is created pursuant to the ...
83,1962,Albania,Economic,The Albanian regime introduced an austerity pr...
84,1962,France,Independence,"End of the Algerian War, Algeria, a French col..."
85,1963,Bulgary,Geopolitical,Pirin Macedonia was declared by Zhivkov as a p...


In [9]:
# Encontre os anos mínimos e máximos no dataframe
min_year_ev = ev.year_event.min()
max_year_ev = ev.year_event.max()

### **B.1) Primeira vista boa**

In [15]:
del app1

NameError: name 'app1' is not defined

In [16]:
app1 = JupyterDash(__name__)

In [24]:
app1.layout = html.Div([
    html.Div([
        html.P("Select up to 3 countries:"),
        dcc.Dropdown(
            id="dropdown-country",
            options=[{'label': str(val), 'value': val} for val in var.country.unique()],
            value=[],
            multi=True,
        ),
         html.Div(id="checkboxes-container-country")
    ]),

    html.Div([
        html.P("Select 1 indicator:"),
        dcc.Dropdown(
            id="dropdown-indicator",
            options=[{'label': val, 'value': val} for val in var.indicator.unique()],
            value=[],
            multi=True,
        ),
        html.Div(id="checkboxes-container-indicator")
    ]),

    html.Div([
        html.Div([
            html.P("Enter the start year:"),
            dcc.Input(
                id='start-year1',
                type='number',
                value=min_year,
                style={'width': '100px'}
                )
            ], style={'width': '24%', 'display': 'inline-block', 'padding': '0 20'}),
        html.Div([
            html.P("Enter the end year:"),
            dcc.Input(
                id='end-year1',
                type='number',
                value=max_year,
                style={'width': '100px'}
                ),
            ], style={'width': '24%', 'display': 'inline-block', 'padding': '0 20'}),
        dcc.Graph(id="bar1"),
        dcc.Graph(id='map1', figure={}),
        dcc.Graph(id="line1")
    ], style={'width': '48%', 'display': 'inline-block', 'padding': '0 20'}),

    html.Div([
        html.Div([
            html.P("Enter the start year:"),
            dcc.Input(
                id='start-year2',
                type='number',
                value=min_year,
                style={'width': '100px'}
                )
            ], style={'width': '24%', 'display': 'inline-block', 'padding': '0 20'}),
        html.Div([
            html.P("Enter the end year:"),
            dcc.Input(
                id='end-year2',
                type='number',
                value=max_year,
                style={'width': '100px'}
                ),
            ], style={'width': '24%', 'display': 'inline-block', 'padding': '0 20'}),
        dcc.Graph(id="bar2"),
        dcc.Graph(id='map2', figure={}),
        dcc.Graph(id="line2")
        ], style={'width': '48%', 'display': 'inline-block', 'padding': '0 20'})
])

@app1.callback(Output("dropdown-country", "value"), [Input("dropdown-country", "value")])
def validate_country_selection(value):
    if len(value) > 3:
        return value[:3]  # seleciona apenas os primeiros 3 países da lista
    return value

@app1.callback(Output("dropdown-indicator", "value"), [Input("dropdown-indicator", "value")])
def validate_indicator_selection(value):
    if len(value) > 1:
        return value[:1]  # seleciona apenas o primeiro 1 indicador da lista
    return value


# GRÁFICOS DO LADO ESQUERDO DA PÁGINA

@app1.callback(Output("bar1", "figure"), 
              [Input("dropdown-country", "value"), Input("dropdown-indicator", "value"), Input("start-year1", "value"), Input("end-year1", "value")])
def display_bar1(selected_countries, selected_indicators, start_year, end_year):
    if not selected_countries and not selected_indicators:
        return {}
    filtered_var = var[(var['country'].isin(selected_countries)) & (var['indicator'].isin(selected_indicators)) & (var['year'] >= start_year) & (var['year'] <= end_year)]
    print(filtered_var)
    grouped_var = round(filtered_var.groupby(['country', 'indicator']).mean(),2).reset_index()
    bar1 = px.bar(grouped_var, x='country', y='value', text='value', color='country', title='Média da variável - Período 1')
    bar1.update_layout(title_x=0.5)
    return bar1

@app1.callback(Output('map1', 'figure'),
              [Input("dropdown-country", "value"), Input("dropdown-indicator", "value"), Input("start-year1", "value"), Input("end-year1", "value")],
               allow_duplicate=True)
def display_map1(selected_countries, selected_indicators, start_year, end_year):
    if not selected_countries:
        return {}
    
    filtered_var = var[var['country'].isin(selected_countries) & (var['indicator'].isin(selected_indicators)) & (var['year'] >= start_year) & (var['year'] <= end_year)]
    # Cria um novo DataFrame apenas com as informações do primeiro país selecionado
    ref_country_var = filtered_var[filtered_var['country'] == selected_countries[0]]
    # Calcula a média dos valores para o primeiro país selecionado
    ref_country_mean = ref_country_var['value'].mean()
    # Cria uma nova coluna com o valor percentual em relação ao primeiro país
    filtered_var['value_pct'] = (filtered_var['value'] / ref_country_mean - 1) * 100
    # Cria o mapa
    grouped_var = (round(filtered_var.groupby(['country', 'indicator', 'country_code'])['value_pct'].mean(), 2).reset_index())
    map1 = px.choropleth(grouped_var, 
                         locations="country_code",
                         color="value_pct",
                         hover_name="country",
                         projection="kavrayskiy7",
                         scope='europe',
                         title="Variação percentual em relação a " + selected_countries[0] + " - Período 1",
                         #range_color=[-50, 50],
                         color_continuous_scale='RdYlGn',
                         color_continuous_midpoint=0)
    map1.update_layout(height=500)
    map1.update_layout(title_x=0.5)

    # Verifica se há valores NaN para cada país selecionado pelo usuário
    if any(var[(var['country'].isin(selected_countries)) & (var['indicator'].isin(selected_indicators)) & (var['year'] >= start_year) & (var['year'] <= end_year)]['value'].isnull()):
        # Cria uma lista com os códigos dos países sem dados
        no_data_country_codes = [filtered_var.groupby('country_code')['value'].mean().isna().idxmax()]
        # Adiciona uma camada com a cor cinza e a legenda "não há dados" para os países sem dados
        no_data_layer = go.Choropleth(locations=no_data_country_codes,
                                      z=[np.nan]*len(no_data_country_codes),
                                      hoverinfo='skip',
                                      showscale=False,
                                      colorscale=[[0, 'rgb(220,220,220)'], [1, 'rgb(220,220,220)']])
        map1.add_trace(no_data_layer)
    return map1

@app1.callback(Output("line1", "figure"), 
              [Input("dropdown-country", "value"), Input("dropdown-indicator", "value"), Input("start-year1", "value"), Input("end-year1", "value")])
def display_line1(selected_countries, selected_indicators, start_year, end_year):
    if not selected_countries and not selected_indicators:
        return {}
    filtered_var = var[(var['country'].isin(selected_countries)) & (var['indicator'].isin(selected_indicators)) & (var['year'] >= start_year) & (var['year'] <= end_year)]
    line1 = px.line(filtered_var, x='year', y='value', color='country', orientation='h', title='Linha do tempo da variável - Período 1')
    line1.update_layout(title_x=0.5)
    return line1


# GRÁFICOS DO LADO DIREITO DA PÁGINA

@app1.callback(Output("bar2", "figure"), 
              [Input("dropdown-country", "value"), Input("dropdown-indicator", "value"), Input("start-year2", "value"), Input("end-year2", "value")])
def display_bar2(selected_countries, selected_indicators, start_year, end_year):
    if not selected_countries and not selected_indicators:
        return {}
    filtered_var = var[(var['country'].isin(selected_countries)) & (var['indicator'].isin(selected_indicators)) & (var['year'] >= start_year) & (var['year'] <= end_year)]
    grouped_var = round(filtered_var.groupby(['country', 'indicator']).mean(),2).reset_index()
    bar2 = px.bar(grouped_var, x='country', y='value', text='value', color='country', title='Média da variável - Período 2')
    bar2.update_layout(title_x=0.5)
    return bar2

@app1.callback(Output('map2', 'figure'),
              [Input("dropdown-country", "value"), Input("dropdown-indicator", "value"), Input("start-year2", "value"), Input("end-year2", "value")],
               allow_duplicate=True)
def display_map2(selected_countries, selected_indicators, start_year, end_year):
    if not selected_countries:
        return {}    
    filtered_var = var[var['country'].isin(selected_countries) & (var['indicator'].isin(selected_indicators)) & (var['year'] >= start_year) & (var['year'] <= end_year)]
    # Cria um novo DataFrame apenas com as informações do primeiro país selecionado
    ref_country_var = filtered_var[filtered_var['country'] == selected_countries[0]]
    # Calcula a média dos valores para o primeiro país selecionado
    ref_country_mean = ref_country_var['value'].mean()
    # Cria uma nova coluna com o valor percentual em relação ao primeiro país
    filtered_var['value_pct'] = (filtered_var['value'] / ref_country_mean - 1) * 100
    # Lida com valores nulos
    filtered_var.fillna(value=np.nan, inplace=True)
    # Cria o mapa
    grouped_var = (round(filtered_var.groupby(['country', 'indicator', 'country_code'])['value_pct'].mean(), 2).reset_index())
    map2 = px.choropleth(grouped_var, 
                         locations="country_code",
                         color="value_pct",
                         hover_name="country",
                         projection="kavrayskiy7",
                         scope='europe',
                         title="Variação percentual em relação a " + selected_countries[0] + " - Período 2",
                         #range_color=[-50, 50],
                         color_continuous_scale='RdYlGn',
                         color_continuous_midpoint=0,
                         color_discrete_map={'np.nan': 'gray'})
    map2.update_layout(height=500)
    map2.update_layout(title_x=0.5)
    return map2

@app1.callback(Output("line2", "figure"),
              [Input("dropdown-country", "value"), Input("dropdown-indicator", "value"), Input("start-year2", "value"), Input("end-year2", "value")])
def display_line2(selected_countries, selected_indicators, start_year, end_year):
    if not selected_countries and not selected_indicators:
        return {}
    filtered_var = var[(var['country'].isin(selected_countries)) & (var['indicator'].isin(selected_indicators)) & (var['year'] >= start_year) & (var['year'] <= end_year)]
    line2 = px.line(filtered_var, x='year', y='value', color='country', title='Linha do tempo da variável - Período 2')
    line2.update_layout(title_x=0.5)
    return line2

if __name__ == "__main__":
    app1.run_server(port=8051)

Dash is running on http://127.0.0.1:8051/

Dash app running on http://127.0.0.1:8051/
Empty DataFrame
Columns: [country, country_code, indicator, year, value]
Index: []
Empty DataFrame
Columns: [country, indicator, year, value]
Index: []
Empty DataFrame
Columns: [country, country_code, indicator, year, value]
Index: []
Empty DataFrame
Columns: [country, indicator, year, value]
Index: []
      country country_code       indicator  year     value
2418  Austria          AUT  gdp_per_capita  1960    935.46
2419  Austria          AUT  gdp_per_capita  1961   1031.82
2420  Austria          AUT  gdp_per_capita  1962   1087.83
2421  Austria          AUT  gdp_per_capita  1963   1167.00
2422  Austria          AUT  gdp_per_capita  1964   1269.41
...       ...          ...             ...   ...       ...
2599  Belgium          BEL  gdp_per_capita  2017  44198.48
2600  Belgium          BEL  gdp_per_capita  2018  47544.98
2601  Belgium          BEL  gdp_per_capita  2019  46638.68
2602  Belgium       

In [11]:
del app2

NameError: name 'app2' is not defined

In [179]:
app2 = JupyterDash(__name__, suppress_callback_exceptions=True)

In [180]:
#estilo = ['https://fonts.googleapis.com/css?family=Trebuchet+MS']
#app2 = JupyterDash(__name__, external_stylesheets = estilo)

In [181]:
### PÁGINA 1
# Criando o HTML geral
page_1_layout = html.Div([
    html.H2('Welcome to the country comparator!'),
    
    html.Div([
        html.P("Select up to 5 countries:"),
        dcc.Dropdown(
            id="dropdown-country",
            options=[{'label': str(val), 'value': val} for val in sorted(var.country.unique())],
            value=[],
            multi=True,
        ),
         html.Div(id="checkboxes-container-country")
    ]),

    html.Div([
        html.P("Select 1 indicator:"),
        dcc.Dropdown(
            id="dropdown-indicator",
            options=[{'label': val, 'value': val} for val in sorted(var.indicator.unique())],
            value=[],
            multi=True,
        ),
        html.Div(id="checkboxes-container-indicator")
    ]),

    html.Div([
        html.Div([
            html.P("Enter the start year:"),
            dcc.Input(
                id='start-year1',
                type='number',
                value=min_year,
                style={'width': '100px'}
                )
        ], style={'width': '24%', 'display': 'inline-block', 'padding': '0 20'}),
        
        html.Div([
            html.P("Enter the end year:"),
            dcc.Input(
                id='end-year1',
                type='number',
                value=max_year,
                style={'width': '100px'}
                ),
        ], style={'width': '24%', 'display': 'inline-block', 'padding': '0 20'}),
        
        dcc.Graph(id="line1"),
        dcc.Graph(id="bar1"),
        dcc.Graph(id='map1', figure={})
        ], style={'width': '48%', 'display': 'inline-block', 'padding': '0 20'}),

    html.Div([
        html.Div([
            html.P("Enter the start year:"),
            dcc.Input(
                id='start-year2',
                type='number',
                value=min_year,
                style={'width': '100px'}
                )
        ], style={'width': '24%', 'display': 'inline-block', 'padding': '0 20'}),
        html.Div([
            html.P("Enter the end year:"),
            dcc.Input(
                id='end-year2',
                type='number',
                value=max_year,
                style={'width': '100px'}
                ),
        ], style={'width': '24%', 'display': 'inline-block', 'padding': '0 20'}),
        
        dcc.Graph(id="line2"),
        dcc.Graph(id="bar2"),
        dcc.Graph(id='map2', figure={})
        ], style={'width': '48%', 'display': 'inline-block', 'padding': '0 20'})
])



#### FUNÇÕES DA PÁGINA 1
@app2.callback(Output("dropdown-country", "value"), [Input("dropdown-country", "value")])
def validate_country_selection(value):
    if len(value) > 5:
        return value[:5]  # seleciona apenas os primeiros 5 países da lista
    return value

@app2.callback(Output("dropdown-indicator", "value"), [Input("dropdown-indicator", "value")])
def validate_indicator_selection(value):
    if len(value) > 1:
        return value[:1]  # seleciona apenas o primeiro 1 indicador da lista
    return value


# Gráficos do lado esquerdo
@app2.callback(Output("line1", "figure"), 
              [Input("dropdown-country", "value"), Input("dropdown-indicator", "value"), Input("start-year1", "value"), Input("end-year1", "value")])
def display_line1(selected_countries, selected_indicators, start_year, end_year):
    if not selected_countries and not selected_indicators:
        return {}
    filtered_var = var[(var['country'].isin(selected_countries))
                       & (var['indicator'].isin(selected_indicators))
                       & (var['year'] >= start_year) & (var['year'] <= end_year)]
    line1 = px.line(filtered_var,
                    x='year',
                    y='value',
                    color='country',
                    title='Linha do tempo - Período 1')
    line1.update_layout(title_x=0.5)
    return line1


@app2.callback(Output("bar1", "figure"), 
              [Input("dropdown-country", "value"), Input("dropdown-indicator", "value"), Input("start-year1", "value"), Input("end-year1", "value")])
def display_bar1(selected_countries, selected_indicators, start_year, end_year):
    if not selected_countries and not selected_indicators:
        return {}
    filtered_var = var[(var['country'].isin(selected_countries))
                       & (var['indicator'].isin(selected_indicators))
                       & (var['year'] >= start_year) & (var['year'] <= end_year)]
    grouped_var = round(filtered_var.groupby(['country', 'indicator']).mean(),2).reset_index()
    if selected_indicators:
        title = "Variação percentual em relação a " + selected_indicators[0] + " - Período 1"
    else:
        title = "Variação percentual - Período 1"
    bar1 = px.bar(grouped_var,
                  x='value',
                  y='country',
                  text='value',
                  color='country',
                  orientation='h',
                  title=title)
    bar1.update_layout(title_x=0.5)
    return bar1


@app2.callback(Output('map1', 'figure'),
              [Input("dropdown-country", "value"), Input("dropdown-indicator", "value"), Input("start-year1", "value"), Input("end-year1", "value")],
               allow_duplicate=True)
def display_map1(selected_countries, selected_indicators, start_year, end_year):
    if not selected_countries:
        return {}
    
    filtered_var = var[var['country'].isin(selected_countries) & (var['indicator'].isin(selected_indicators)) & (var['year'] >= start_year) & (var['year'] <= end_year)]
    # Cria um novo DataFrame apenas com as informações do primeiro país selecionado
    ref_country_var = filtered_var[filtered_var['country'] == selected_countries[0]]
    # Calcula a média dos valores para o primeiro país selecionado
    ref_country_mean = ref_country_var['value'].mean()
    # Cria uma nova coluna com o valor percentual em relação ao primeiro país
    filtered_var['value_pct'] = (filtered_var['value'] / ref_country_mean - 1) * 100
    # Cria o mapa
    grouped_var = (round(filtered_var.groupby(['country', 'indicator', 'country_code'])['value_pct'].mean(), 2).reset_index())
    map1 = px.choropleth(grouped_var, 
                         locations="country_code",
                         color="value_pct",
                         hover_name="country",
                         projection="kavrayskiy7",
                         scope='europe',
                         title="Variação percentual em relação a " + selected_countries[0] + " - Período 1",
                         #range_color=[-50, 50],
                         color_continuous_scale='RdYlGn',
                         color_continuous_midpoint=0)
    map1.update_layout(height=500)
    map1.update_layout(title_x=0.5)

    # Verifica se há valores NaN para cada país selecionado pelo usuário
    if any(var[(var['country'].isin(selected_countries)) &
               (var['indicator'].isin(selected_indicators)) &
               (var['year'] >= start_year) & (var['year'] <= end_year)]['value'].isnull()):
        # Cria uma lista com os códigos dos países sem dados
        no_data_country_codes = [filtered_var.groupby('country_code')['value'].mean().isna().idxmax()]
        # Adiciona uma camada com a cor cinza e a legenda "não há dados" para os países sem dados
        no_data_layer = go.Choropleth(locations=no_data_country_codes,
                                      z=[np.nan]*len(no_data_country_codes),
                                      hoverinfo='skip',
                                      showscale=False,
                                      colorscale=[[0, 'rgb(220,220,220)'], [1, 'rgb(220,220,220)']])
        map1.add_trace(no_data_layer)
    return map1


# Gráficos do lado direito
@app2.callback(Output("line2", "figure"),
              [Input("dropdown-country", "value"), Input("dropdown-indicator", "value"), Input("start-year2", "value"), Input("end-year2", "value")])
def display_line2(selected_countries, selected_indicators, start_year, end_year):
    if not selected_countries and not selected_indicators:
        return {}
    filtered_var = var[(var['country'].isin(selected_countries)) &
                       (var['indicator'].isin(selected_indicators)) &
                       (var['year'] >= start_year) & (var['year'] <= end_year)]
    line2 = px.line(filtered_var,
                    x='year',
                    y='value',
                    color='country',
                    title='Linha do tempo - Período 2')
    line2.update_layout(title_x=0.5)
    return line2


@app2.callback(Output("bar2", "figure"), 
              [Input("dropdown-country", "value"), Input("dropdown-indicator", "value"), Input("start-year2", "value"), Input("end-year2", "value")])
def display_bar2(selected_countries, selected_indicators, start_year, end_year):
    if not selected_countries and not selected_indicators:
        return {}
    filtered_var = var[(var['country'].isin(selected_countries)) &
                       (var['indicator'].isin(selected_indicators)) &
                       (var['year'] >= start_year) & (var['year'] <= end_year)]
    grouped_var = round(filtered_var.groupby(['country', 'indicator']).mean(),2).reset_index()
    if selected_indicators:
        title = "Variação percentual em relação a " + selected_indicators[0] + " - Período 1"
    else:
        title = "Variação percentual - Período 1"
    bar2 = px.bar(grouped_var,
                  x='value',
                  y='country',
                  text='value',
                  color='country',
                  orientation='h',
                  title=title)
    bar2.update_layout(title_x=0.5)
    return bar2


@app2.callback(Output('map2', 'figure'),
              [Input("dropdown-country", "value"), Input("dropdown-indicator", "value"), Input("start-year2", "value"), Input("end-year2", "value")],
               allow_duplicate=True)
def display_map2(selected_countries, selected_indicators, start_year, end_year):
    if not selected_countries:
        return {}    
    filtered_var = var[var['country'].isin(selected_countries) & (var['indicator'].isin(selected_indicators)) & (var['year'] >= start_year) & (var['year'] <= end_year)]
    # Cria um novo DataFrame apenas com as informações do primeiro país selecionado
    ref_country_var = filtered_var[filtered_var['country'] == selected_countries[0]]
    # Calcula a média dos valores para o primeiro país selecionado
    ref_country_mean = ref_country_var['value'].mean()
    # Cria uma nova coluna com o valor percentual em relação ao primeiro país
    filtered_var['value_pct'] = (filtered_var['value'] / ref_country_mean - 1) * 100
    # Lida com valores nulos
    filtered_var.fillna(value=np.nan, inplace=True)
    # Cria o mapa
    grouped_var = (round(filtered_var.groupby(['country', 'indicator', 'country_code'])['value_pct'].mean(), 2).reset_index())
    map2 = px.choropleth(grouped_var, 
                         locations="country_code",
                         color="value_pct",
                         hover_name="country",
                         projection="kavrayskiy7",
                         scope='europe',
                         title="Variação percentual em relação a " + selected_countries[0] + " - Período 2",
                         #range_color=[-50, 50],
                         color_continuous_scale='RdYlGn',
                         color_continuous_midpoint=0,
                         color_discrete_map={'np.nan': 'gray'})
    map2.update_layout(height=500)
    map2.update_layout(title_x=0.5)
    return map2



### PÁGINA 
# Definir o estilo da tabela
table_style = {
    'fontFamily': 'Arial',
    'borderCollapse': 'collapse',
    'fontSize': '12px',
    'lineHeight': '1.2',
    'marginBottom': '10px',
    'width': '100%',
    'height': '50%'
}

cell_style = {
    'border': '1.1px solid #d3d3d3',
    'padding': '6px',
    'textAlign': 'left'
}

# HTML
page_2_layout = html.Div([
    html.H2('What happened before/after some event?'),
    
    html.Div([
        html.P("Select 1 reference country:"),
        dcc.Dropdown(
            id="dropdown-ref_country",
            options=[{'label': str(val), 'value': val} for val in ev.country_event.unique()],
            value=[],
            multi=True)
    ], style={'width': '15%', 'display': 'inline-block', 'padding': '0 20'}),
    
    html.Div([
        html.P("Select some event type(s):"),
        dcc.Dropdown(
            id='dropdown-event_type',
            options=[{'label': str(val), 'value': val} for val in ev.event_type.unique()],
            value=[],
            multi=True)
    ], style={'width': '85%', 'display': 'inline-block', 'padding': '0 20'}),
    
    html.Div([
        dcc.RangeSlider(
            id='year-slider2',
            min=min_year_ev,
            max=max_year_ev,
            value=[2000, 2010],
            marks={str(year): {'label': str(year), 'style': {'transform': 'rotate(90deg)'}} for year in range(min_year_ev, max_year_ev+1)},
            step=None)
    ]),

    html.Div(
        id='table-container',
        children=generate_table(ev)
    )
])


#### FUNÇÕES DA PÁGINA 2
@app2.callback(Output("dropdown-ref_country", "value"), [Input("dropdown-ref_country", "value")])
def validate_ref_country_selection(value):
    if len(value) > 1:
        return value[:1]
    return value


@app2.callback(Output("dropdown-event_type", "value"), [Input("dropdown-event_type", "value")])
def validate_event_type_selection(value):
    if len(value) > 14:
        return value[:14]
    return value

def generate_table(dataframe):
    return html.Table(
        # Header
        [html.Tr([html.Th(col, style=cell_style) for col in dataframe.columns], style=table_style)] +
        # Body
        [html.Tr([
            html.Td(dataframe.iloc[i][col], style=cell_style) for col in dataframe.columns
        ], style=table_style) for i in range(len(dataframe))],
        style=table_style
    )


@app2.callback(Output('table-container', 'children'),
               [Input('dropdown-ref_country','value'), Input('dropdown-event_type','value'), Input('year-slider2', 'value')],
               allow_duplicate=True)
def update_table(ref_country, event_type, year_range):
    # Filtrar o dataframe original utilizando os valores selecionados nos dropdowns e no RangeSlider
    filtered_df = ev[(ev['country_event'].isin(ref_country)) & 
                     (ev['event_type'].isin(event_type)) & 
                     (ev['year'].between(year_range[0], year_range[1]))]
    # Chamar a função generate_table para criar a tabela HTML
    table = generate_table(filtered_df)
    # Retornar a tabela HTML
    return table




### LAYOUT GERAL
# Criando o HTML geral
tabs_layout = dcc.Tabs(id='tabs', value='tab-1', children=[
    dcc.Tab(label='COUNTRY COMPARATOR', value='tab-1', children=[page_1_layout]),
    dcc.Tab(label='IMPACT OF EVENTS', value='tab-2', children=[page_2_layout]),
])

app2.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Div([
        html.Img(src='infocountry_logo.png', style={'width':'362px', 'height':'77px', 'display': 'inline-block'}),
    ], style={'width':'362px', 'height':'77px', 'display': 'inline-block'}),

    html.Div([tabs_layout]),
    
    html.Div(id='page-content')
], style={'fontFamily':'Arial'})


@app2.callback(Output('page-content', 'children'),
              Input('url', 'pathname'))
def display_page(pathname):
    if pathname == '/page-2':
        return page_2_layout
    else:
        pass


# Gráficos do lado esquerdo
@app2.callback(Output("line3", "figure"), 
              [Input("dropdown-country", "value"), Input("dropdown-indicator", "value"), Input("start-year2", "value"), Input("end-year2", "value")])
def display_line1(selected_countries, selected_indicators, start_year, end_year):
    if not selected_countries and not selected_indicators:
        return {}
    filtered_var = var[(var['country'].isin(selected_countries))
                       & (var['indicator'].isin(selected_indicators))
                       & (var['year'] >= start_year) & (var['year'] <= end_year)]
    line1 = px.line(filtered_var,
                    x='year',
                    y='value',
                    color='country',
                    title='Linha do tempo - Período 1')
    line1.update_layout(title_x=0.5)
    return line1


# Gráficos do lado direito
@app2.callback(Output("line2", "figure"),
              [Input("dropdown-country", "value"), Input("dropdown-indicator", "value"), Input("start-year2", "value"), Input("end-year2", "value")])
def display_line2(selected_countries, selected_indicators, start_year, end_year):
    if not selected_countries and not selected_indicators:
        return {}
    filtered_var = var[(var['country'].isin(selected_countries)) &
                       (var['indicator'].isin(selected_indicators)) &
                       (var['year'] >= start_year) & (var['year'] <= end_year)]
    line2 = px.line(filtered_var,
                    x='year',
                    y='value',
                    color='country',
                    title='Linha do tempo - Período 2')
    line2.update_layout(title_x=0.5)
    return line2


### RODAR TUDO
if __name__ == "__main__":
    app2.run_server(port=8051, debug=True)

Dash is running on http://127.0.0.1:8051/

Dash app running on http://127.0.0.1:8051/


In [19]:
del app3

NameError: name 'app3' is not defined

In [20]:
app3 = JupyterDash(__name__, suppress_callback_exceptions=True)

In [194]:
### PÁGINA 
# Definir o estilo da tabela
table_style = {
    'fontFamily': 'Arial',
    'borderCollapse': 'collapse',
    'fontSize': '12px',
    'lineHeight': '1.2',
    'marginBottom': '10px',
    'width': '100%',
    'height': '50%'
}

cell_style = {
    'border': '1.1px solid #d3d3d3',
    'padding': '6px',
    'textAlign': 'left'
}

def generate_table(dataframe):
    return html.Table(
        # Header
        [html.Tr([html.Th(col, style=cell_style) for col in dataframe.columns], style=table_style)] +
        # Body
        [html.Tr([
            html.Td(dataframe.iloc[i][col], style=cell_style) for col in dataframe.columns
        ], style=table_style) for i in range(len(dataframe))],
        style=table_style
    )


# HTML
app3.layout = html.Div([
    html.Iframe(src='codigo.js', width='100%', height='600'),

    html.H2('Select info that you want!'),
    
    html.Div([
        html.P("Select 1 reference country:"),
        dcc.Dropdown(
            id="dropdown-ref_country",
            options=[{'label': str(val), 'value': val} for val in sorted(ev.country_event.unique())],
            value=[],
            multi=True)
    ], style={'width': '15%', 'display': 'inline-block', 'padding': '0 20'}),
    
    html.Div([
        html.P("Select some event type(s):"),
        dcc.Dropdown(
            id='dropdown-event_type',
            options=[{'label': str(val), 'value': val} for val in sorted(ev.event_type.unique())],
            value=[],
            multi=True)
    ], style={'width': '85%', 'display': 'inline-block', 'padding': '0 20'}),
    
    html.P("You filter the events in time too:"),
    html.Div([
        dcc.RangeSlider(
            id='year-slider2',
            min=min_year_ev,
            max=max_year_ev,
            value=[min_year_ev, max_year_ev],
            marks={str(year): {'label': str(year), 'style': {'fontFamily':'Arial', 'fontSize':'12px', 'transform':'rotate(45deg)'}}
                   if year%2==1 else '' for year in range(min_year_ev, max_year_ev+1, 1)},
            step=None)
    ]),

    html.P("Main events that happened:"),
    html.Div(id='table-container',
             children=generate_table(ev)),

    html.H2('What happened before/after some event?'),
    
    html.Div([
        html.P("Select the reference country + up to 4 to compare:"),
        dcc.Dropdown(
            id="dropdown-country-ev",
            options=[{'label': str(val), 'value': val} for val in sorted(var.country.unique())],
            value=[],
            multi=True,
        ),
         html.Div(id="checkboxes-container-country-ev")
    ], style={'width': '72%', 'display': 'inline-block', 'padding': '0 20'}),

    html.Div([
        html.P("Select 1 indicator:"),
        dcc.Dropdown(
            id="dropdown-indicator-ev",
            options=[{'label': val, 'value': val} for val in sorted(var.indicator.unique())],
            value=[],
            multi=True,
        ),
        html.Div(id="checkboxes-container-indicator-ev")
    ], style={'width': '24%', 'display': 'inline-block', 'padding': '0 20'}),

    html.Div([
        html.Div([
            html.P("Enter the reference year:"),
            dcc.Input(
                id='reference-year',
                type='number',
                value=min_year,
                style={'width': '100px'}
                )
        ], style={'width': '24%', 'display': 'inline-block', 'padding': '60 20'}),

        html.Div([
            html.P("Enter interval years:"),
            dcc.Input(
                id='interval-years',
                type='number',
                value=10,
                style={'width': '100px'}
                ),
        ], style={'width': '24%', 'display': 'inline-block', 'padding': '0 20'})
    ]),
    
    html.Div([
        html.Div([
            dcc.Graph(id="line3"),
            dcc.Graph(id="bar3"),
            dcc.Graph(id='map3', figure={})
        ], style={'width': '48%', 'display': 'inline-block', 'padding': '0 20'}),

        html.Div([
            dcc.Graph(id="line4"),
            dcc.Graph(id="bar4"),
            dcc.Graph(id='map4', figure={})
        ], style={'width': '48%', 'display': 'inline-block', 'padding': '0 20'})
    ])
])


#### FUNÇÕES DA PÁGINA 
@app3.callback(Output("dropdown-ref_country", "value"), [Input("dropdown-ref_country", "value")])
def validate_ref_country_selection(value):
    if len(value) > 1:
        return value[:1]
    return value


@app3.callback(Output("dropdown-event_type", "value"), [Input("dropdown-event_type", "value")])
def validate_event_type_selection(value):
    if len(value) > 14:
        return value[:14]
    return value

# Tabela
@app3.callback(Output('table-container', 'children'),
               [Input('dropdown-ref_country','value'), Input('dropdown-event_type','value'), Input('year-slider2', 'value')],
               allow_duplicate=True)
def update_table(ref_country, event_type, year_range):
    # Filtrar o dataframe original utilizando os valores selecionados nos dropdowns e no RangeSlider
    filtered_df = ev[(ev['country_event'].isin(ref_country)) & 
                     (ev['event_type'].isin(event_type)) & 
                     (ev['year_event'].between(year_range[0], year_range[1]))]
    # Chamar a função generate_table para criar a tabela HTML
    table = generate_table(filtered_df)
    # Retornar a tabela HTML
    return table


@app3.callback(Output("dropdown-country-ev", "value"), [Input("dropdown-country-ev", "value")])
def validate_country_selection(value):
    if len(value) > 5:
        return value[:5]  # seleciona apenas os primeiros 5 países da lista
    return value


@app3.callback(Output("dropdown-indicator-ev", "value"), [Input("dropdown-indicator-ev", "value")])
def validate_indicator_selection(value):
    if len(value) > 1:
        return value[:1]  # seleciona apenas o primeiro 1 indicador da lista
    return value


# Gráficos do lado esquerdo
@app3.callback(Output("line3", "figure"), 
              [Input("dropdown-ref_country", "value"), Input("dropdown-country-ev", "value"), Input("dropdown-indicator-ev", "value"),
               Input("reference-year", "value"), Input("interval-years", "value")],
               allow_duplicate=True)
def display_line3(ref_country, selected_countries, selected_indicators, reference_year, interval_years):
    if not selected_countries or not selected_indicators or not ref_country:
        return {}
    
    if ref_country not in selected_countries:
        selected_countries.append(ref_country)
    else:
        pass
    
    filtered_var = var[(var['country'].isin(selected_countries)) &
                       (var['indicator'].isin(selected_indicators)) &
                       (var['year'] >= int(reference_year)-int(interval_years)) &
                       (var['year'] <= int(reference_year)+int(interval_years))]
    
    if filtered_var.empty:
        return {}
    
    line3 = px.line(filtered_var,
                    x='year',
                    y='value',
                    color='country',
                    title='Linha do tempo - Período 1')
    line3.update_layout(title_x=0.5)
    return line3
      


### RODAR TUDO
if __name__ == "__main__":
    app3.run_server(port=8051, debug=True)

Dash is running on http://127.0.0.1:8051/

Dash app running on http://127.0.0.1:8051/


### **B.4) Display avançado**

In [254]:
del app4

In [255]:
app4 = JupyterDash(__name__, suppress_callback_exceptions=True)

In [256]:
### PÁGINA
table_style = {
    'fontFamily': 'Arial',
    'borderCollapse': 'collapse',
    'fontSize': '12px',
    'lineHeight': '1.2',
    'marginBottom': '10px',
    'width': '100%',
    'height': '50%',
    'tableLayout': 'auto'
}

cell_style = {
    'border': '1.1px solid #d3d3d3',
    'padding': '6px',
    'textAlign': 'left'
}

def generate_table(dataframe):
    return html.Table(
        # Header
        [html.Tr([html.Th(col, style=cell_style) for col in dataframe.columns], style=table_style)] +
        # Body
        [html.Tr([
            html.Td(dataframe.iloc[i][col], style=cell_style) for col in dataframe.columns
        ], style=table_style) for i in range(len(dataframe))],
        style=table_style
    )


# Criando o HTML geral
app4.layout = html.Div([

    html.Br(style={"height": '50'}),

    html.Div([
        html.Img(src='assets/infocountry_logo.png'),
    ]),
    
    html.Div([
        html.H2('Welcome to the country comparator!')
    ]),
    
    html.Div([
        html.P("Select up to 5 countries:"),
        dcc.Dropdown(
            id="dropdown-country",
            options=[{'label': str(val), 'value': val} for val in sorted(var.country.unique())],
            value=[],
            multi=True),
    ], style={'width': '72%', 'display': 'inline-block', 'padding': '0 20'}),
    
    html.Div([
        html.P("Select 1 indicator:"),
        dcc.Dropdown(
            id="dropdown-indicator",
            options=[{'label': val, 'value': val} for val in sorted(var.indicator_user.unique())],
            value=[],
            multi=True),
    ], style={'width': '28%', 'display': 'inline-block', 'padding': '0 20'}),
    
    html.Div([
        dcc.Graph(id="line")
    ], style={'width': '80%', 'margin':'auto'}),

    html.Br(style={"height": '25'}),
    html.Hr(),
    html.Br(style={"height": '25'}),

    html.Div([
        html.H2('What relevant events happened?'),
        ]),
    
    html.Div([
        html.Div([
            html.P("Select 1 reference country:"),
            dcc.Dropdown(
                id="dropdown-ref_country",
                options=[{'label': str(val), 'value': val} for val in sorted(ev.country_event.unique())],
                value=[],
                multi=True)
        ], style={'width': '22%', 'display': 'inline-block', 'padding': '0 20'}),

        html.Div([
            html.P("Select some event type(s):"),
            dcc.Dropdown(
                id='dropdown-event_type',
                options=[{'label': 'SELECT ALL', 'value': 'all'}] + [{'label': str(val), 'value': val} for val in sorted(ev.event_type.unique())],
                value=[],
                multi=True)
        ], style={'width': '78%', 'display': 'inline-block', 'padding': '0 20'}),

        html.Br(style={"height": '100'}),

        html.Div([
            html.P("Select a range of interest:"),
            dcc.RangeSlider(
                id='year-slider3',
                min=min_year_ev,
                max=max_year_ev,
                value=[min_year_ev, max_year_ev],
                marks={str(year): {'label': str(year), 'style': {'transform':'rotate(45deg)'}}
                       if year%2==1 else '' for year in range(min_year_ev, max_year_ev+1, 1)},
                step=None)
        ], style={'margin':'auto'}),

        html.Div([
            html.P(f"Main events happened:"),
            html.Div(id='table-container', children=generate_table(ev))
        ], style={'width':'92%', 'margin':'auto'})
    ]),

    html.Br(style={"height": '50'}),
    html.Hr(),
    html.Br(style={"height": '25'}),

    html.Div([
        html.H2('Mean impact before and after some event!'),
    ]),

    html.Div([
        html.Div([
            html.P("Enter the reference year:"),
            dcc.Input(
                id='reference-year',
                type='number',
                value=2000, #min_year_ev,
                style={'width': '100px'}
                )
        ], style={'width': '20%', 'display': 'inline-block', 'padding': '60 20'}),

        html.Div([
            html.P("Enter interval years:"),
            dcc.Input(
                id='interval-years',
                type='number',
                value=10,
                style={'width': '100px'}
                ),
        ], style={'width': '20%', 'display': 'inline-block', 'padding': '0 20'})
    ]),
    
    html.Br(style={"height": '50'}),

    html.Div([
        html.Div([
            dcc.Graph(id="bar1"),
            dcc.Graph(id='map1', figure={})
        ], style={'width':'50%', 'display':'inline-block', 'padding':'0 20'}),
    
        html.Div([
            dcc.Graph(id="bar2"),
            dcc.Graph(id='map2', figure={})
        ], style={'width':'50%', 'display':'inline-block', 'padding':'0 20'})
    ]),

    html.Br(style={"height": '100'})
    
], style={'fontFamily':'Arial', 'fontSize':'90%', 'width':'90%', 'margin':'auto', 'align-items':'center'})



#### FUNÇÕES
# Seleciona apenas os primeiros 5 países da lista
@app4.callback(Output("dropdown-country", "value"), [Input("dropdown-country", "value")], allow_duplicate=True)
def validate_country_selection(value):
    if len(value) > 5:
        return value[:5]
    return value


# Seleciona apenas o primeiro 1 indicador da lista
@app4.callback(Output("dropdown-indicator", "value"), [Input("dropdown-indicator", "value")], allow_duplicate=True)
def validate_indicator_selection(value):
    if len(value) > 1:
        return value[:1]  
    return value


# Primeiro gráfico - Linhas
@app4.callback(Output("line", "figure"), 
              [Input("dropdown-country", "value"), Input("dropdown-indicator", "value")], allow_duplicate=True)
def display_line1(selected_countries, selected_indicators):
    if not selected_countries and not selected_indicators:
        return {}
    filtered_var = var[(var['country'].isin(selected_countries)) & (var['indicator_user'].isin(selected_indicators))]
    line1 = px.line(filtered_var,
                    x='year',
                    y='value',
                    color='country',
                    title='Variables time line')
    line1.update_layout(title_x=0.5,
                        title_y=0.93,
                        margin=dict(l=50, r=50, t=50, b=50),
                        height=400,
                        yaxis=dict(title='Value'),
                        xaxis=dict(title='Year'),
                        legend_title_text='Countries') 
    return line1


# Seletor de país referência (a partir dos primeiros selecionados)
@app4.callback(Output('dropdown-ref_country', 'options'), [Input('dropdown-country', 'value')], allow_duplicate=True)
def update_ref_country_options(selected_countries):
    options = [{'label': str(val), 'value': val} for val in sorted(ev.country_event.unique())]
    if selected_countries:
        options = [opt for opt in options if opt['value'] in selected_countries]
    return options


# Escolhe o primeiro da lista acima
@app4.callback(Output("dropdown-ref_country", "value"), [Input("dropdown-ref_country", "value")], allow_duplicate=True)
def validate_ref_country_selection(value):
    if len(value) > 1:
        return value[:1]
    return value


## Seletor de eventos
@app4.callback(Output('dropdown-event_type', 'value'),
               [Input('dropdown-event_type', 'value'), Input('dropdown-event_type', 'options')],
               allow_duplicate=True)
def update_event_type(value, options):
    ctx = dash.callback_context
    if not ctx.triggered:
        button_id = 'None'
    else:
        button_id = ctx.triggered[0]['prop_id'].split('.')[0]
    if button_id == 'dropdown-event_type':
        if len(value) > 14:
            return value[:14]
        return value
    elif button_id == 'dropdown-event_type':
        return [option['value'] for option in options] if 'all' in [option['value'] for option in options] else []
    else:
        return dash.no_update

    
## Tabela
@app4.callback(Output('table-container', 'children'),
               [Input('dropdown-ref_country','value'), Input('dropdown-event_type','value'), Input('year-slider3', 'value'),
                Input('dropdown-event_type', 'options')],
               allow_duplicate=True)
def update_table(ref_country, event_type, year_range, event_type_options):
    # Verificar se "SELECT ALL" está selecionado
    if 'all' in event_type:
        # Obter todos os valores de evento disponíveis, exceto "all"
        event_type_values = [option['value'] for option in event_type_options if option['value'] != 'all']
        # Usar todos os valores de evento disponíveis se nenhum evento individual tiver sido selecionado
        event_type = event_type_values if len(event_type) == 0 else event_type_values
    # Filtrar o dataframe original utilizando os valores selecionados nos dropdowns e no RangeSlider
    filtered_df_ev = ev[(ev['country_event'].isin(ref_country)) & 
                       (ev['event_type'].isin(event_type)) & 
                       (ev['year_event'].between(year_range[0], year_range[1]))]
    # Chamar a função generate_table para criar a tabela HTML
    table_ev = generate_table(filtered_df_ev)
    # Retornar a tabela HTML
    return table_ev


## Gráficos do lado esquerdo
# Barra 1
@app4.callback(Output("bar1", "figure"),
              [Input("dropdown-country", "value"), Input("dropdown-indicator", "value"), Input("reference-year", "value"), 
               Input("interval-years", "value"), Input("dropdown-ref_country", "value")],
               allow_duplicate=True)
def display_bar1(selected_countries, selected_indicators, reference_year, interval_years, reference_country):
    if not reference_country:
        return {}
    # Identificar se cada linha corresponde ao país de referência
    filtered_var = var[(var['country'].isin(selected_countries)) &
                       (var['indicator_user'].isin(selected_indicators)) &
                       (var['year'] >= int(reference_year)-int(interval_years)) &
                       (var['year'] <= int(reference_year))]
    # Variável "referência"
    filtered_var['is_reference'] = filtered_var['country'].map(lambda x: 1 if x == str(reference_country[0]) else 0)
    # "Novo" dataframe
    grouped_var = round(filtered_var.groupby(['country', 'indicator_user']).mean(),2).reset_index()
    # Ordenar o dataframe para a referência ser a primeira barra
    grouped_var = grouped_var.sort_values(by=['is_reference', 'value'], ascending=[False, True])
    # Criando gráfico
    bar1 = px.bar(grouped_var,
                  x='country',
                  y='value',
                  text='value',
                  color='country',
                  orientation='v',
                  title="Mean value - BEFORE year reference")
    bar1.update_layout(title_x=0.5,
                       title_y=0.93,
                       margin=dict(l=50, r=50, t=50, b=50),
                       height=400,
                       yaxis=dict(title='Country'),
                       xaxis=dict(title='Mean value'),
                       legend_title_text='Countries') 
    return bar1

# Mapa 1
@app4.callback(Output('map1', 'figure'),
              [Input("dropdown-country", "value"), Input("dropdown-indicator", "value"), Input("reference-year", "value"), 
               Input("interval-years", "value"), Input("dropdown-ref_country", "value")],
               allow_duplicate=True)
def display_map1(selected_countries, selected_indicators, reference_year, interval_years, reference_country):
    if not selected_countries or not reference_country:
        return {}
    filtered_var = var[var['country'].isin(selected_countries) &
                       (var['indicator_user'].isin(selected_indicators)) &
                       (var['year'] >= int(reference_year)-int(interval_years)) &
                       (var['year'] <= int(reference_year))]
    # Cria um novo DataFrame apenas com as informações do país de referência selecionado
    ref_country_var = filtered_var[filtered_var['country'] == str(reference_country[0])]
    # Calcula a média dos valores para o país de referência selecionado
    ref_country_mean = ref_country_var['value'].mean()
    # Cria uma nova coluna com o valor percentual em relação ao país de referência selecionado
    xxx = (round(filtered_var.groupby(['country', 'indicator_user', 'country_code'])['value'].mean(), 2).reset_index())
    # Add valores percentuais ao dataframe de comparação
    xxx['value_pct'] = (xxx['value']/ref_country_mean - 1) * 100
    # Criando mapa
    map1 = px.choropleth(xxx,
                         locations="country_code",
                         color="value_pct",
                         locationmode='ISO-3',
                         hover_name="country",
                         projection="kavrayskiy7",
                         scope='europe',
                         color_continuous_scale='RdYlGn',
                         color_continuous_midpoint=0,
                         color_discrete_map={'np.nan': 'gray'},
                         labels={'value_pct': '% Variation'})
    # Personalizar o layout do mapa
    map1.update_layout(geo=dict(scope='europe',
                                showland=True,
                                landcolor='#f2f2f2',
                                showocean=True,
                                oceancolor='#a6bddb',
                                showcountries=True,
                                countrycolor='gray',
                                showsubunits=True,
                                subunitcolor='gray',
                                projection=dict(type='kavrayskiy7')),
                                title = {'text': "Percentual variation related to " + str(reference_country[0]) + "<br>BEFORE year reference",
                                         'x': 0.5,
                                         'y': 0.85,
                                         'xanchor': 'center',
                                         'yanchor': 'top'})
    return map1


## Gráficos do lado direito
# Barras 2
@app4.callback(Output("bar2", "figure"),
              [Input("dropdown-country", "value"), Input("dropdown-indicator", "value"), Input("reference-year", "value"), 
               Input("interval-years", "value"), Input("dropdown-ref_country", "value")],
               allow_duplicate=True)
def display_bar2(selected_countries, selected_indicators, reference_year, interval_years, reference_country):
    if not reference_country:
        return {}
    # Identificar se cada linha corresponde ao país de referência
    filtered_var = var[(var['country'].isin(selected_countries)) &
                       (var['indicator_user'].isin(selected_indicators)) &
                       (var['year'] >= int(reference_year)) &
                       (var['year'] <= int(reference_year)+int(interval_years))]
    # Variável "referência"
    filtered_var['is_reference'] = filtered_var['country'].map(lambda x: 1 if x == str(reference_country[0]) else 0)
    # "Novo" dataframe
    grouped_var = round(filtered_var.groupby(['country', 'indicator_user']).mean(),2).reset_index()
    # Ordenar o dataframe para a referência ser a primeira barra
    grouped_var = grouped_var.sort_values(by=['is_reference', 'value'], ascending=[False, True])
    # Criando gráfico
    bar2 = px.bar(grouped_var,
                  x='country',
                  y='value',
                  text='value',
                  color='country',
                  orientation='v',
                  title="Mean value - AFTER year reference")
    bar2.update_layout(title_x=0.5,
                       title_y=0.93,
                       margin=dict(l=50, r=50, t=50, b=50),
                       height=400,
                       yaxis=dict(title='Country'),
                       xaxis=dict(title='Mean value'),
                       legend_title_text='Countries') 
    return bar2


# Mapa 2
@app4.callback(Output('map2', 'figure'),
              [Input("dropdown-country", "value"), Input("dropdown-indicator", "value"), Input("reference-year", "value"), 
               Input("interval-years", "value"), Input("dropdown-ref_country", "value")],
               allow_duplicate=True)
def display_map2(selected_countries, selected_indicators, reference_year, interval_years, reference_country):
    if not selected_countries or not reference_country:
        return {}    
    filtered_var = var[var['country'].isin(selected_countries) &
                       (var['indicator_user'].isin(selected_indicators)) &
                       (var['year'] >= int(reference_year)) &
                       (var['year'] <= int(reference_year)+int(interval_years))]
    # Cria um novo DataFrame apenas com as informações do país de referência selecionado
    ref_country_var = filtered_var[filtered_var['country'] == str(reference_country[0])]
    # Calcula a média dos valores para o país de referência selecionado
    ref_country_mean = ref_country_var['value'].mean()
    # Cria uma nova coluna com o valor percentual em relação ao país de referência selecionado
    xxx = (round(filtered_var.groupby(['country', 'indicator_user', 'country_code'])['value'].mean(), 2).reset_index())
    # Add valores percentuais ao dataframe de comparação
    xxx['value_pct'] = (xxx['value']/ref_country_mean - 1) * 100
    # Criando mapa
    map2 = px.choropleth(xxx,
                         locations="country_code",
                         color="value_pct",
                         locationmode='ISO-3',
                         hover_name="country",
                         projection="kavrayskiy7",
                         scope='europe',
                         color_continuous_scale='RdYlGn',
                         color_continuous_midpoint=0,
                         color_discrete_map={'np.nan': 'gray'},
                         labels={'value_pct': '% Variation'})
    # Personalizar o layout do mapa
    map2.update_layout(geo=dict(scope='europe',
                                showland=True,
                                landcolor='#f2f2f2',
                                showocean=True,
                                oceancolor='#a6bddb',
                                showcountries=True,
                                countrycolor='gray',
                                showsubunits=True,
                                subunitcolor='gray',
                                projection=dict(type='kavrayskiy7')),
                                title = {'text': "Percentual variation related to " + str(reference_country[0]) + "<br>AFTER year reference",
                                         'x': 0.5,
                                         'y': 0.85,
                                         'xanchor': 'center',
                                         'yanchor': 'top'})
    return map2


### RODAR TUDO
if __name__ == "__main__":
    app4.run_server(port=8051, debug=True)

Dash is running on http://127.0.0.1:8051/

Dash app running on http://127.0.0.1:8051/


In [188]:
filtered_var = var[var['country'].isin(['Austria','Belarus']) &
                       (var['indicator'].isin(['inflation'])) &
                       (var['year'] >= 2000-10) &
                       (var['year'] <= 2000)]

In [189]:
filtered_var

,country,country_code,indicator,indicator_user,year,value
4804,Austria,AUT,inflation,Inflation (%),1990,3.26
4805,Austria,AUT,inflation,Inflation (%),1991,3.34
4806,Austria,AUT,inflation,Inflation (%),1992,4.02
4807,Austria,AUT,inflation,Inflation (%),1993,3.63
4808,Austria,AUT,inflation,Inflation (%),1994,2.95
4809,Austria,AUT,inflation,Inflation (%),1995,2.24
4810,Austria,AUT,inflation,Inflation (%),1996,1.86
4811,Austria,AUT,inflation,Inflation (%),1997,1.31
4812,Austria,AUT,inflation,Inflation (%),1998,0.92
4813,Austria,AUT,inflation,Inflation (%),1999,0.57


In [191]:
filtered_var['is_reference'] = filtered_var['country'].map(lambda x: 1 if x == 'Austria' else 0)

In [193]:
grouped_var = round(filtered_var.groupby(['country', 'indicator_user']).mean(),2).reset_index()

In [194]:
grouped_var

,country,indicator_user,year,value,is_reference
0,Austria,Inflation (%),1995.0,2.40,1.0
1,Belarus,Inflation (%),1995.0,596.55,0.0


In [115]:
ref_country_var = filtered_var[filtered_var['country'] == 'Austria']

In [116]:
ref_country_var

,country,country_code,indicator,indicator_user,year,value
4804,Austria,AUT,inflation,Inflation (%),1990,3.26
4805,Austria,AUT,inflation,Inflation (%),1991,3.34
4806,Austria,AUT,inflation,Inflation (%),1992,4.02
4807,Austria,AUT,inflation,Inflation (%),1993,3.63
4808,Austria,AUT,inflation,Inflation (%),1994,2.95
4809,Austria,AUT,inflation,Inflation (%),1995,2.24
4810,Austria,AUT,inflation,Inflation (%),1996,1.86
4811,Austria,AUT,inflation,Inflation (%),1997,1.31
4812,Austria,AUT,inflation,Inflation (%),1998,0.92
4813,Austria,AUT,inflation,Inflation (%),1999,0.57


In [29]:
ref_country_var.country.nunique()

1

In [38]:
ref_country_var['value'].mean()

2.4036363636363633


In [130]:
xxx = (round(filtered_var.groupby(['country', 'indicator_user', 'country_code'])['value'].mean(), 2).reset_index())

In [131]:
xxx

,country,indicator_user,country_code,value
0,Austria,Inflation (%),AUT,2.40
1,Belarus,Inflation (%),BLR,596.55


In [133]:
xxx

,country,indicator_user,country_code,value,value_pct
0,Austria,Inflation (%),AUT,2.40,0.00
1,Belarus,Inflation (%),BLR,596.55,24756.25


In [120]:
filtered_var['value']

4804       3.26
4805       3.34
4806       4.02
4807       3.63
4808       2.95
4809       2.24
4810       1.86
4811       1.31
4812       0.92
4813       0.57
4814       2.34
4866        NaN
4867        NaN
4868        NaN
4869    1190.23
4870    2221.02
4871     709.35
4872      52.71
4873      63.94
4874      72.87
4875     293.68
4876     168.62
Name: value, dtype: float64

In [121]:
filtered_var['value_pct'] = (filtered_var['value'] / ref_country_mean - 1) * 100

In [123]:
filtered_var['value_pct']

4804       35.627837
4805       38.956127
4806       67.246596
4807       51.021180
4808       22.730711
4809       -6.807867
4810      -22.617247
4811      -45.499244
4812      -61.724660
4813      -76.285930
4814       -2.647504
4866             NaN
4867             NaN
4868             NaN
4869    49417.889561
4870    92302.496218
4871    29411.535552
4872     2092.927383
4873     2560.136157
4874     2931.656581
4875    12118.154312
4876     6915.204236
Name: value_pct, dtype: float64

In [124]:
grouped_var = (round(filtered_var.groupby(['country', 'indicator_user', 'country_code'])['value_pct'].mean(), 2).reset_index())

In [125]:
grouped_var

,country,indicator_user,country_code,value_pct
0,Austria,Inflation (%),AUT,0.00
1,Belarus,Inflation (%),BLR,24718.75


In [70]:
print(grouped_var['country_code'].isnull().sum())
print(set(grouped_var['country_code']) - set(px.data.election_geojson()['features']['id']))

0


TypeError: list indices must be integers or slices, not str